In [1]:
%reload_ext autoreload

In [5]:
# Packages needed

#!pip install "tabulate" "pandas_summary" "imblearn" "fastai" "joblib"
#!pip install fastai==0.7.0

In [6]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
# Constants
PROBABILITY_CUTOFF = 0.50

# Utility functions
def plot_roc_pr(m, X_valid, y_valid):
    # Generate the probabilities
    #y_pred_prob = generate_predictions(X_valid)
    y_pred_prob = m.predict_proba(X_valid)
    y_pred_prob = y_pred_prob[:, 1]

    # Calculate the roc metrics
    fpr, tpr, thresholds = roc_curve(y_valid, y_pred_prob)

    fig, ax = plt.subplots(figsize=(20,10))

    # Plot the ROC curve
    plt.plot(fpr,tpr, label='ROC')

    # Add labels and diagonal line
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.plot([0, 1], [0, 1], "k--", label='no skill')

    # Plot a precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_prob)
    area_under_curve = auc(recall, precision)
    ap = average_precision_score(y_valid, y_pred_prob)

    # plot no skill
    pyplot.plot([0, 1], [0.5, 0.5], linestyle='--', label='no skill')
    # plot the precision-recall curve for the model
    pyplot.plot(recall, precision, marker='.', label='precision-recall')

    legend = ax.legend(loc='best', shadow=True, fontsize='medium')

    # show the plot
    pyplot.show()
    
    # Output AUC and average precision score
    print('auc=%.3f ap=%.3f' % (area_under_curve, ap))
    
def uber_score(y_valid, validate_predictions):
    return [precision_score(y_valid,validate_predictions), recall_score(y_valid,validate_predictions), f1_score(y_valid,validate_predictions), accuracy_score(y_valid,validate_predictions), cohen_kappa_score(y_valid,validate_predictions), mean_absolute_error(y_valid,validate_predictions)]

def graph_corr(df):
    fig, ax = plt.subplots(figsize=(20,10))
    corr = df.corr()
    sns.heatmap(corr, cmap='YlGnBu', annot_kws={'size':30}, ax=ax)
    ax.set_title("Correlation Matrix", fontsize=14)
    plt.show()
    
def data_summary(df_raw):
    array = []
    for column_name in df_raw.select_dtypes(include=['float64', 'int64']).columns:
        mean = df_raw[column_name].mean()
        median = df_raw[column_name].median()
        std = df_raw[column_name].std()
        cv = df_raw[column_name].std()/df_raw[column_name].mean()
        trimmed_mean = trim_mean(df_raw[column_name].values, 0.1)
        array.append([column_name, mean, median, std, cv, trimmed_mean])
    print(tabulate(array,headers=['Column', 'Mean', 'Median', 'Std', 'cv', 'Trimmed Mean']))

def generate_predictions(X_valid, cutoff=PROBABILITY_CUTOFF):
    return m.predict(X_valid)
    #return (m.predict_proba(X_valid)[:,1] >= cutoff).astype(bool)
    
def conf_matrix(y_valid, validate_predictions):
    cm = confusion_matrix(y_valid, validate_predictions)
    print(cm)
    
def remove_columns_test(df):
    m = RandomForestClassifier(
        n_estimators=10,
        min_samples_leaf=1, 
        max_features='sqrt',
        n_jobs=-1, 
        #oob_score=True,
        max_depth=3,
        bootstrap=False,
        criterion='gini',
        class_weight={0: 2, 1: 1})
    x, _ = split_vals(df, n_trn)
    m.fit(x, y_train)
    y_pred = m.predict(x)
    return uber_score(y_train, y_pred)

In [11]:
!pwd

/Users/justinhennessy/Code/100-days-of-code/project


In [31]:
# Load in data

PATH = "../data/"
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [32]:
def prepare_data(df_raw):
    # Sort data by date
    df_raw = df_raw.sort_values(by='licence_registration_date')
    
    # Convert annual_revenue from a string to a float
    df_raw['annual_revenue'] = pd.to_numeric(df_raw['annual_revenue'].str.replace(',', ''))
    
    # Convert fields to INT and setting any NaNs to the mean of that type
    case_types = ['cases_total','cases_open','cases_closed','cases_age_hours_total','cases_age_hours_average', 'last_login_days']

    for case_type in case_types:
        default_value = df_raw[case_type].fillna(df_raw[case_type].median())
        df_raw[case_type] = df_raw[case_type].fillna(default_value).astype(int)
    
    # Fix missing values for annual revenue, replace with mean/trimmed mean of the plan size they are on
    plan_list = df_raw.plan[~pd.isnull(df_raw.plan)].unique()

    for plan in plan_list:
        mean = round(df_raw.annual_revenue[df_raw.plan == plan].mean(), 2)
        trimmed_mean = trim_mean(df_raw.annual_revenue[df_raw.plan == plan].values, 0.1)
    
        if pd.isnull(mean):
            revenue = 0
        else:
            revenue = mean
        df_raw.loc[df_raw.plan==plan, 'annual_revenue'] = df_raw.loc[df_raw.plan==plan, 'annual_revenue'].fillna(revenue)
        
    # 'bin' last login days

    bins = [1, 3, 7, 14, 30, 60]
    group_names = ['day', 'few_days', 'week', 'fortnight', 'month']

    # need to get the mean of the plan size for last_login_days and set each row to that
    #df_raw.last_login_days = df_raw.last_login_days.fillna(np.mean(df_raw.last_login_days))

    last_login_categories = pd.cut(df_raw['last_login_days'], bins, labels=group_names)
    df_raw['last_login_categories'] = pd.cut(df_raw['last_login_days'], bins, labels=group_names)
    #pd.value_counts(df_raw['last_login_categories'])
    
    # one-hot encode fields
    dummy_columns = ['customer_account_status', 'last_login_categories', 'plan']

    for dummy_column in dummy_columns:
        dummy = pd.get_dummies(df_raw[dummy_column], prefix=dummy_column)
        df_raw = pd.concat([df_raw,dummy], axis=1)
        df_raw = df_raw.drop(columns=dummy_column)
        
    
    # This breaks all the date features up into number columns
    # These steps can only be run once then you need to comment them out
    add_datepart(df_raw, 'licence_registration_date')
    add_datepart(df_raw, 'golive_date')
    
    # Drop columns, some of these create "Data Leakage", some are just to test if it has impact when they are taken out
    df_raw = df_raw.drop(columns=['customer_account_status_Good', 'last_login_concern',
                                  'last_login_days', 'account_status', 'changing_platform', 
                                  'new_platform', 'licence_status', 'canceldate', 
                                  'cancel_details', 'cancel_reason'])
    
    # Set default values for NaN values in NPS
    df_raw.nps = df_raw.nps.fillna(np.nanmean(df_raw.nps))

    # Set NaN to zero
    features = ['churned', 'interactions_total', 'interactions_completed', 'interactions_no_response', 'interactions_no_onboarding', 'interactions_completed_training']

    for feature in features:
        df_raw[feature] = df_raw[feature].fillna(0)
        
    # Complete the transformation of all data into numbers using proc_df and create training dataframes
    train_cats(df_raw)
    
    return df_raw

In [33]:
print(len(df_raw), df_raw.shape)
df_processed = prepare_data(df_raw)
print(len(df_processed), df_processed.shape)

5889 (5889, 38)
5889 (5889, 99)


In [34]:
df_filtered = df_processed[df_processed.licence_registration_Year >2017]
print(len(df_filtered), df_filtered.shape)

1351 (1351, 99)


In [35]:
print(df_filtered.licence_registration_Year.head(), df_filtered.licence_registration_Year.tail())

4538    2018
4539    2018
4540    2018
4541    2018
4542    2018
Name: licence_registration_Year, dtype: int64 5884    2019
5885    2019
5887    2019
5886    2019
5888    2019
Name: licence_registration_Year, dtype: int64


In [36]:
len(df_filtered[df_filtered.churned == 1]), len(df_filtered[df_filtered.churned == 0])

(412, 939)

In [37]:
df_trn, y_trn, nas = proc_df(df_filtered, 'churned')
print(len(df_trn), df_trn.shape)

1351 (1351, 102)


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
def split_vals(a,n): return a[:n], a[n:]

n_valid = 500
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

In [39]:
# Fix the dataset imbalance

sm = SMOTE(random_state=12, ratio = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
X_train_res.shape

(1040, 102)

In [40]:
len(df_filtered[df_filtered.churned == 1]), len(df_filtered[df_filtered.churned == 0])

(412, 939)

In [41]:
m = RandomForestClassifier(
    n_estimators=200,
    min_samples_leaf=2, 
    max_features='sqrt',
    n_jobs=-1, 
    #oob_score=True,
    max_depth=3,
    bootstrap=False,
    criterion='gini',
    #class_weight={0: 2, 1: 1}
    )

m.fit(X_train_res, y_train_res)

# Save to file in the current working directory
joblib_file = "churn_model.pkl"
joblib.dump(m, joblib_file)

['churn_model.pkl']